In [1]:
from utils.spark_session import get_spark_session
from pyspark.sql import SparkSession

spark: SparkSession = get_spark_session()

25/11/15 17:27:37 WARN Utils: Your hostname, DESKTOP-68EC7N1 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/15 17:27:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/dottier/venvs/flights_venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/dottier/.ivy2/cache
The jars for the packages stored in: /home/dottier/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-268d5e78-fd86-47e7-9431-80e6f9c279d8;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 299ms :: artifacts dl 10ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	----------------------------

In [4]:
spark.sql("SHOW TABLES in bronze").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|   bronze| airports|      false|
|   bronze|  flights|      false|
+---------+---------+-----------+



In [3]:
from delta.tables import DeltaTable

bronze_flights = DeltaTable.forName(spark, "bronze.flights")
bronze_flights.history().show(5)

25/10/12 17:28:28 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      1|2025-10-12 17:22:...|  NULL|    NULL|    WRITE|{mode -> Overwrit...|NULL|    NULL|     NULL|          0|  Serializable|        false|{numFiles -> 16, ...|        NULL|Apache-Spark/3.5....|
|      0|2025-10-12 16:13:...|  NULL|    NULL|    WRITE|{mode -> Overwrit...|NULL|    NULL|     NULL|       NULL|  Serializable|        false|{numFiles -> 158,...|        NULL|Apache-Spark/3.5....|
+-------+-

In [4]:
# Roll back the table to version 14
bronze_flights.restoreToVersion(0)


25/10/12 17:29:17 WARN DAGScheduler: Broadcasting large task binary with size 1076.9 KiB
25/10/12 17:29:19 WARN DAGScheduler: Broadcasting large task binary with size 1122.5 KiB
25/10/12 17:29:23 WARN DAGScheduler: Broadcasting large task binary with size 1159.9 KiB


DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [6]:
spark.sql("SHOW TABLES in silver").show()

+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|   silver| dim_aircraft|      false|
|   silver|dim_aircrafts|      false|
|   silver| dim_airlines|      false|
|   silver| dim_airports|      false|
|   silver|  dim_runways|      false|
|   silver|  fct_flights|      false|
+---------+-------------+-----------+



In [7]:
spark.sql("DROP DATABASE silver CASCADE")

25/10/14 06:48:01 WARN TxnHandler: Cannot perform cleanup since metastore table does not exist


DataFrame[]

In [2]:
from utils.io_utils import read_df

silver_flights = read_df(spark, "silver", "fct_flights")
print(silver_flights.count())

silver_runways = read_df(spark, "silver", "dim_runways")
silver_runways.count()

25/11/01 17:40:41 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/11/01 17:40:41 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/11/01 17:40:42 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/11/01 17:40:42 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore dottier@127.0.1.1
25/11/01 17:40:42 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
25/11/01 17:40:44 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


325058


168

In [3]:
silver_runways.count()

168

In [3]:
silver_runways.show(10, truncate=False)


+----------------------------------------------------------------+----------------------------------------------------------------+----------------------------------------------------------------+-----------+------------+--------+------------+---------+-----------+----------+------------------------+---------+---------+----------------------------------------------------------------+--------------------------+--------------------------+--------------------------+------------------+
|runway_version_key                                              |runway_sk                                                       |airport_sk                                                      |runway_name|true_heading|surface |has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet|latitude |longitude|_data_hash                                                      |_ingested_at              |_inserted_at              |effective_start_date      |effective_end_date|
+-------------------------

In [5]:
import pyspark.sql.functions as sf

silver_runways.select(
    sf.count_distinct(sf.col("airport_sk"))
).show()

+--------------------------+
|count(DISTINCT airport_sk)|
+--------------------------+
|                        31|
+--------------------------+



In [4]:
a = silver_flights.dropDuplicates(["flight_composite_pk"])
a.count()

183756

In [9]:
silver_airports = read_df(spark, "silver", "dim_airports")

In [10]:
silver_airports.printSchema()

root
 |-- airport_sk: long (nullable = true)
 |-- airport_iata: string (nullable = true)
 |-- airport_icao: string (nullable = true)
 |-- airport_name: string (nullable = true)
 |-- municipality_name: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- continent_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elevation_feet: double (nullable = true)
 |-- airport_time_zone: string (nullable = true)



In [ ]:
from load.database import transaction_context

with transaction_context() as db:
    db.execute("""
        CREATE TABLE dim_airports (
            airport_sk BIGINT PRIMARY KEY,
            airport_iata VARCHAR(10),
            airport_icao VARCHAR(10),
            airport_name VARCHAR(255),
            municipality_name VARCHAR(255),
            country_name VARCHAR(100),
            continent_name VARCHAR(50),
            latitude DOUBLE PRECISION,
            longitude DOUBLE PRECISION,
            elevation_feet DOUBLE PRECISION,
            airport_time_zone VARCHAR(100)
        );
    """)

ERROR:root:Error in database: relation "dim_airports" already exists
Traceback (most recent call last):
  File "/home/dottier/flights_etl/load/database.py", line 26, in get_db
    yield cur
  File "/tmp/ipykernel_1758/4159816736.py", line 4, in <module>
    db.execute("""
psycopg2.errors.DuplicateTable: relation "dim_airports" already exists



In [ ]:
with transaction_context() as db:
    db.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
    result = db.fetchone()
    print(result)

('dim_airports',)


In [2]:
from utils.io_utils import read_df
from load.utils import load_table_to_postgres

silver_airports = read_df(spark, "silver", "dim_airports")
silver_runways = read_df(spark, "silver", "dim_runways")
silver_airlines = read_df(spark, "silver", "dim_airlines")
silver_aircrafts = read_df(spark, "silver", "dim_aircrafts")
silver_flights = read_df(spark, "silver", "fct_flights")
silver_regions = read_df(spark, "silver", "dim_regions", optional=True)

25/11/15 17:27:55 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/11/15 17:27:55 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/11/15 17:27:58 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/11/15 17:27:58 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore dottier@127.0.1.1
25/11/15 17:27:59 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
25/11/15 17:28:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
silver_airports.printSchema()

root
 |-- airport_bk: string (nullable = true)
 |-- airport_iata: string (nullable = true)
 |-- airport_icao: string (nullable = true)
 |-- airport_name: string (nullable = true)
 |-- municipality_name: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elevation_feet: double (nullable = true)
 |-- airport_time_zone: string (nullable = true)
 |-- _ingested_at: timestamp (nullable = true)
 |-- _inserted_at: timestamp (nullable = true)
 |-- _data_hash: string (nullable = true)
 |-- airport_gps: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- airport_local_code: string (nullable = true)
 |-- airport_type: string (nullable = true)
 |-- scheduled_service: boolean (nullable = true)
 |-- iso_country: string (nullable = true)



In [3]:
silver_flights.printSchema()

root
 |-- flight_composite_pk: string (nullable = true)
 |-- departure_airport_bk: string (nullable = true)
 |-- arrival_airport_bk: string (nullable = true)
 |-- airline_bk: string (nullable = true)
 |-- aircraft_bk: string (nullable = true)
 |-- departure_runway_version_bk: string (nullable = true)
 |-- arrival_runway_version_bk: string (nullable = true)
 |-- departure_runway_name: string (nullable = true)
 |-- arrival_runway_name: string (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- flight_callsign: string (nullable = true)
 |-- flight_status: string (nullable = true)
 |-- codeshare_status: string (nullable = true)
 |-- is_cargo: boolean (nullable = true)
 |-- dep_has_basic: boolean (nullable = true)
 |-- dep_has_live: boolean (nullable = true)
 |-- arr_has_basic: boolean (nullable = true)
 |-- arr_has_live: boolean (nullable = true)
 |-- quality_desc: string (nullable = true)
 |-- dep_scheduled_at_utc: timestamp (nullable = true)
 |-- dep_revised_at_utc: times

In [3]:
import pyspark.sql.functions as sf


In [5]:
silver_flights.filter(sf.col("departure_airport_bk").isNull()).show()

+-------------------+--------------------+------------------+----------+-----------+---------------------------+-------------------------+---------------------+-------------------+-------------+---------------+-------------+----------------+--------+-------------+------------+-------------+------------+------------+--------------------+------------------+-----------------+--------------------------+------------------------+-----------------------+------------------+--------------------+------------------+-----------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+------------+--------+----------------+----------+-----------------------------+--------+---------+-----------+----------------+--------------+------------+------------+--------------+
|flight_composite_pk|departure_airport_bk|arrival_airport_bk|airline_bk|aircraft_bk|departure_runway_version_bk|arrival_runway_version_bk|depa

In [6]:
import pyspark.sql.functions as sf

silver_runways.groupBy("runway_version_bk").agg(
    sf.count("*").alias("count")    
).filter(sf.col("count") > 1).show(truncate=False)

+----------------------------------------------------------------+-----+
|runway_version_bk                                               |count|
+----------------------------------------------------------------+-----+
|787e4305c0bba11e8ad5f7c22634cec5253d9da203cd515ed61dcee2c7bbb55c|2    |
|83d3815a1756b383e41bd1236dc1a3c7a6ac2947d6007de200986accd4f86a00|2    |
|ff2b7de7aaf1dde976d2cddc822695e7d38ebe9b24cf8376b25d576167ed9e78|2    |
|432a52f969fa3cc4e7756368e95f21158ecb1d7d7f2f30e5bd767798b5e121a3|2    |
|6c212119edfd2d712d7b5bc42e966f3130642396bd645aee6fccdd5c2f79bc3f|2    |
|615493041fcb9b4f4778c192fe5759f07f2fbb4d82ae6275b3dce6126fe6b97c|2    |
|766a40f9a1af25e488c8e2453bda7701faa827d05d532e5ff63805305d6deca5|2    |
+----------------------------------------------------------------+-----+



In [7]:
silver_runways.filter(sf.col("runway_version_bk") == "787e4305c0bba11e8ad5f7c22634cec5253d9da203cd515ed61dcee2c7bbb55c").show(20, truncate=False)

+----------------------------------------------------------------+----------------------------------------------------------------+----------------------------------------------------------------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------------+--------------------------+--------------------------+------------------+---------------+--------------+
|runway_version_bk                                               |runway_bk                                                       |airport_bk                                                      |runway_name|true_heading|surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet|latitude|longitude|_ingested_at              |_inserted_at              |effective_start_date      |effective_end_date|runway_end_type|elevation_feet|
+----------------------------------------------------------------+--------------------------------

In [17]:
silver_runways.filter((col("length_feet") == 4300) & (col("width_feet") == 150) & (col("runway_name") == "22") & (col("elevation_feet") == 692)).show(10)

+--------------------+--------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------+--------------------+--------------------+------------------+---------------+--------------+
|   runway_version_bk|           runway_bk|airport_bk|runway_name|true_heading|surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet|latitude|longitude|        _ingested_at|        _inserted_at|effective_start_date|effective_end_date|runway_end_type|elevation_feet|
+--------------------+--------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------+--------------------+--------------------+------------------+---------------+--------------+
|000fcf5df35840a4a...|fcd2a9f24818851b3...|      NULL|         22|        NULL|    CON|        true|     true|     4300.0|     

In [3]:
silver_runways.count()

179

In [3]:
silver_regions.show(10, truncate=False)

+-----------+----------+--------------------------+--------------+-----------+--------------------+-------------------------+--------------------------+
|region_code|local_code|region_name               |continent_code|iso_country|country_name        |_ingested_at             |_inserted_at              |
+-----------+----------+--------------------------+--------------+-----------+--------------------+-------------------------+--------------------------+
|AD-02      |02        |Canillo Parish            |EU            |AD         |Andorra             |2025-11-10 09:47:43.34118|2025-11-10 09:47:59.446059|
|AD-03      |03        |Encamp Parish             |EU            |AD         |Andorra             |2025-11-10 09:47:43.34118|2025-11-10 09:47:59.446059|
|AD-04      |04        |La Massana Parish         |EU            |AD         |Andorra             |2025-11-10 09:47:43.34118|2025-11-10 09:47:59.446059|
|AD-05      |05        |Ordino Parish             |EU            |AD         |Ando

In [ ]:
from pyspark.sql import functions as sf

In [ ]:
silver_airports.filter(sf.col("iso_country").isNotNull() & sf.col("iso_"))

In [2]:
from delta.tables import DeltaTable

silver_runways_delta = DeltaTable.forName(spark, "silver.dim_runways")

25/11/07 21:32:49 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/11/07 21:32:49 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/11/07 21:32:51 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/11/07 21:32:51 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore dottier@127.0.1.1
25/11/07 21:32:52 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
25/11/07 21:32:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
silver_runways_delta.history().show()
silver_runways_delta.restoreToVersion(0)

+-------+--------------------+------+--------+-----------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|  operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+-----------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      8|2025-11-07 21:30:...|  NULL|    NULL|      MERGE|{predicate -> ["(...|NULL|    NULL|     NULL|          7|  Serializable|        false|{numTargetRowsCop...|        NULL|Apache-Spark/3.5....|
|      7|2025-11-07 21:30:...|  NULL|    NULL|      MERGE|{predicate -> ["(...|NULL|    NULL|     NULL|          6|  Serializable|        false|{numTargetRowsCop...|        NULL|Apache-Spark/3.5....|


25/11/07 21:33:11 WARN DAGScheduler: Broadcasting large task binary with size 1090.9 KiB


DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [3]:
from pyspark.sql import functions as sf

silver_airports.orderBy(sf.col("_inserted_at").desc()).show(10, truncate=False)

+----------------------------------------------------------------+------------+------------+-----------------------------------------+-----------------+------------+------------------+-------------+--------------+-----------------+--------------------------+--------------------------+----------------------------------------------------------------+-----------+--------------+-----------------+-----------+------------------+----------+
|airport_bk                                                      |airport_iata|airport_icao|airport_name                             |municipality_name|country_name|latitude          |longitude    |elevation_feet|airport_time_zone|_ingested_at              |_inserted_at              |_data_hash                                                      |airport_gps|airport_type  |scheduled_service|iso_country|airport_local_code|iso_region|
+----------------------------------------------------------------+------------+------------+--------------------------------

In [4]:
silver_runways.orderBy(sf.col("_inserted_at").desc()).show(10, truncate=False)

+----------------------------------------------------------------+----------------------------------------------------------------+----------------------------------------------------------------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+---------+----------+--------------------------+--------------------------+--------------------------+------------------+--------------+---------------+
|runway_version_bk                                               |runway_bk                                                       |airport_bk                                                      |runway_name|true_heading|surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet|latitude |longitude |_ingested_at              |_inserted_at              |effective_start_date      |effective_end_date|elevation_feet|runway_end_type|
+----------------------------------------------------------------+----------------------------

In [6]:
silver_runways.printSchema()

root
 |-- runway_version_key: string (nullable = true)
 |-- runway_sk: string (nullable = true)
 |-- airport_sk: string (nullable = true)
 |-- runway_name: string (nullable = true)
 |-- true_heading: double (nullable = true)
 |-- surface: string (nullable = true)
 |-- has_lighting: boolean (nullable = true)
 |-- is_closed: boolean (nullable = true)
 |-- length_feet: double (nullable = true)
 |-- width_feet: double (nullable = true)
 |-- displaced_threshold_feet: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- _ingested_at: timestamp (nullable = true)
 |-- _inserted_at: timestamp (nullable = true)
 |-- effective_start_date: timestamp (nullable = true)
 |-- effective_end_date: timestamp (nullable = true)
 |-- runway_end_type: string (nullable = true)
 |-- elevation_feet: integer (nullable = true)



In [3]:
silver_airports.printSchema()

root
 |-- airport_sk: string (nullable = true)
 |-- airport_iata: string (nullable = true)
 |-- airport_icao: string (nullable = true)
 |-- airport_name: string (nullable = true)
 |-- municipality_name: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- continent_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elevation_feet: double (nullable = true)
 |-- airport_time_zone: string (nullable = true)
 |-- _ingested_at: timestamp (nullable = true)
 |-- _inserted_at: timestamp (nullable = true)



In [7]:
silver_flights.printSchema()

root
 |-- flight_composite_pk: string (nullable = true)
 |-- departure_airport_sk: string (nullable = true)
 |-- arrival_airport_sk: string (nullable = true)
 |-- airline_sk: string (nullable = true)
 |-- aircraft_sk: string (nullable = true)
 |-- departure_runway_sk: string (nullable = true)
 |-- arrival_runway_sk: string (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- flight_callsign: string (nullable = true)
 |-- flight_status: string (nullable = true)
 |-- codeshare_status: string (nullable = true)
 |-- is_cargo: boolean (nullable = true)
 |-- dep_scheduled_at_utc: timestamp (nullable = true)
 |-- dep_revised_at_utc: timestamp (nullable = true)
 |-- dep_runway_at_utc: timestamp (nullable = true)
 |-- dep_scheduled_at_local_ntz: timestamp (nullable = true)
 |-- dep_revised_at_local_ntz: timestamp (nullable = true)
 |-- dep_runway_at_local_ntz: timestamp (nullable = true)
 |-- dep_local_timezone: string (nullable = true)
 |-- arr_scheduled_at_utc: timestamp (nulla

In [8]:
silver_flights.filter("departure_runway_sk IS NOT NULL").show(50)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+-------------+------------+--------+----------------+----------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------------+--------------------+--------------+
| flight_composite_pk|departure_airport_sk|  arrival_airport_sk|          airline_sk|         aircraft_sk| departure_runway_sk|   arrival_runway_sk|flight_number|flight_callsign|flight_status

In [ ]:
silver_runways

In [6]:
dim_airports_sql = f"""
    CREATE TABLE dim_airports (
        airport_sk BIGINT PRIMARY KEY,
        airport_iata VARCHAR(10),
        airport_icao VARCHAR(10),
        airport_name VARCHAR(255),
        municipality_name VARCHAR(255),
        country_name VARCHAR(100),
        continent_name VARCHAR(50),
        latitude DOUBLE PRECISION,
        longitude DOUBLE PRECISION,
        elevation_feet DOUBLE PRECISION,
        airport_time_zone VARCHAR(100)
    );
"""

dim_airlines_sql = f"""
    CREATE TABLE dim_airlines (
        airline_sk BIGINT PRIMARY KEY,
        airline_iata VARCHAR(10),
        airline_icao VARCHAR(10),
        airline_name VARCHAR(255)
    );
"""

dim_aircrafts_sql = f"""
    CREATE TABLE dim_aircrafts (
        aircraft_sk BIGINT PRIMARY KEY,
        aircraft_reg VARCHAR(100),
        aircraft_mode_s VARCHAR(100),
        aircraft_model VARCHAR(100),
        aircraft_sk_type VARCHAR(100)
    );
"""

dim_runways_sql = f"""
    CREATE TABLE dim_runways (
        runway_sk BIGINT PRIMARY KEY,
        airport_sk BIGINT REFERENCES dim_airports (airport_sk),
        runway_name VARCHAR(100),
        true_heading DOUBLE PRECISION,
        surface VARCHAR(100),
        has_lighting BOOLEAN,
        is_closed BOOLEAN,
        length_feet DOUBLE PRECISION,
        width_feet DOUBLE PRECISION,
        displaced_threshold_feet DOUBLE PRECISION,
        latitude DOUBLE PRECISION,
        longitude DOUBLE PRECISION,
        effective_start_Date TIMESTAMP,
        effective_end_date TIMESTAMP
    );
"""

dim_flight_details_sql = f"""
    CREATE TABLE dim_flight_details (
        flight_details_sk SERIAL PRIMARY KEY,
        flight_status TEXT NOT NULL,
        codeshare_status TEXT NOT NULL,
        is_cargo BOOLEAN NOT NULL,
        
        -- This ensures we only store each unique combination once
        UNIQUE (flight_status, codeshare_status, is_cargo)
    );
"""

dim_quality_sql = f"""
    CREATE TABLE dim_quality_combination (
        quality_combo_sk SERIAL PRIMARY KEY,
        dep_has_basic BOOLEAN NOT NULL,
        dep_has_live BOOLEAN NOT NULL,
        arr_has_basic BOOLEAN NOT NULL,
        arr_has_live BOOLEAN NOT NULL,

        quality_desc TEXT,

        -- Unique constraint on the flags
        UNIQUE (dep_has_basic, dep_has_live, arr_has_basic, arr_has_live)
    );
"""


fct_flights_sql = f"""
    CREATE TABLE fct_flights (
        -- Surrogate Key for performance
        fact_flight_id BIGSERIAL PRIMARY KEY,
        
        -- Natural Key for business logic & upserts
        flight_composite_pk TEXT NOT NULL,

        -- === FOREIGN KEYS ===
        flight_details_sk INT REFERENCES dim_flight_details(flight_details_sk), -- The new FK
        departure_airport_sk BIGINT REFERENCES dim_airports(airport_sk),
        arrival_airport_sk BIGINT REFERENCES dim_airports(airport_sk),
        airline_sk BIGINT REFERENCES dim_airlines(airline_sk),
        aircraft_sk BIGINT REFERENCES dim_aircrafts(aircraft_sk),
        departure_runway_sk BIGINT REFERENCES dim_runways(runway_sk),
        arrival_runway_sk BIGINT REFERENCES dim_runways(runway_sk),
        quality_combo_sk INT REFERENCES dim_quality_combination(quality_combo_sk),

        -- === DEGENERATE DIMENSIONS (Attributes that stay in the fact table) ===
        flight_number TEXT,      -- High cardinality, belongs here
        flight_callsign TEXT,    -- High cardinality, belongs here
        dep_terminal TEXT,
        arr_gate TEXT,
        dep_checkin_desk TEXT,
        dep_gate TEXT,
        arr_terminal TEXT,
        arr_baggage_belt TEXT,
        dep_local_timezone TEXT,
        arr_local_timezone TEXT,

        -- === MEASURES / FACTS ===
        dep_scheduled_at_utc TIMESTAMP,
        dep_revised_at_utc TIMESTAMP,
        dep_runway_at_utc TIMESTAMP,
        dep_scheduled_at_local_ntz TIMESTAMP,
        dep_revised_at_local_ntz TIMESTAMP,
        dep_runway_at_local_ntz TIMESTAMP,
        arr_scheduled_at_utc TIMESTAMP,
        arr_revised_at_utc TIMESTAMP,
        arr_runway_at_utc TIMESTAMP,
        arr_scheduled_at_local_ntz TIMESTAMP,
        arr_revised_at_local_ntz TIMESTAMP,
        arr_runway_at_local_ntz TIMESTAMP,
        
        last_location_reported_at_utc TIMESTAMP,
        latitude DOUBLE PRECISION,
        longitude DOUBLE PRECISION,
        altitude_ft DOUBLE PRECISION,
        ground_speed_kts DOUBLE PRECISION,
        true_track_deg DOUBLE PRECISION,
        ingestion_hour TEXT,

        -- A unique constraint on the natural key to enforce business uniqueness
        UNIQUE (flight_composite_pk)
    );
"""

In [7]:
from load.utils import transaction_context

with transaction_context() as cur:
    # Create tables with schema
    cur.execute(dim_airports_sql)
    cur.execute(dim_airlines_sql)
    cur.execute(dim_aircrafts_sql)
    cur.execute(dim_runways_sql)
    cur.execute(dim_flight_details_sql)
    cur.execute(dim_quality_sql)
    cur.execute(fct_flights_sql)

ERROR:root:Error in database: relation "dim_airports" already exists
Traceback (most recent call last):
  File "/home/dottier/flights_etl/load/database.py", line 26, in get_db
    yield cur
  File "/home/dottier/flights_etl/load/database.py", line 49, in transaction_context
    yield new_cursor
  File "/tmp/ipykernel_67767/1577498256.py", line 5, in <module>
    cur.execute(dim_airports_sql)
psycopg2.errors.DuplicateTable: relation "dim_airports" already exists



In [ ]:
        airport_sk BIGINT PRIMARY KEY,
        airport_iata VARCHAR(10),
        airport_icao VARCHAR(10),
        airport_name VARCHAR(255),
        municipality_name VARCHAR(255),
        country_name VARCHAR(100),
        continent_name VARCHAR(50),
        latitude DOUBLE PRECISION,
        longitude DOUBLE PRECISION,
        elevation_feet DOUBLE PRECISION,
        airport_time_zone VARCHAR(100)

In [2]:
from load.utils import load_to_postgres, read_df_from_postgres
from load.database import transaction_context

with transaction_context() as master_cursor:
    load_to_postgres(
        spark=spark, df=silver_airports, target_table="dim_airports",
        strategy="upsert", pk_cols=["airport_sk"], cursor=master_cursor
    )
    insert_unknown_airport_sql = f"""
        INSERT INTO dim_airports (airport_sk, airport_iata, airport_icao, airport_name, municipality_name, country_name, continent_name, latitude, longitude, elevation_feet, airport_time_zone)
        VALUES (-1, 'UNK', 'UNK', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 0, 0, 0, 'Unknown')
        ON CONFLICT (airport_sk) DO NOTHING;
    """
    master_cursor.execute(insert_unknown_airport_sql)

    load_to_postgres(
        spark=spark, df=silver_airlines, target_table="dim_airlines",
        strategy="upsert", pk_cols=["airline_sk"], cursor=master_cursor
    )
    load_to_postgres(
        spark=spark, df=silver_aircrafts, target_table="dim_aircrafts",
        strategy="upsert", pk_cols=["aircraft_sk"], cursor=master_cursor
    )

    # unknown_runway = [
    #     {"airport_sk": -1, "runway_name": "UNKNOWN", "true_heading": 0.0, "surface": "Unknown",
    #     "has_lighting": False, "is_closed": False, "length_feet": 0.0, "width_feet": 0.0, "displaced_threshold_feet": 0.0,
    #     "latitude": 0.0, "longitude": 0.0}
    # ]

    load_to_postgres(
        spark=spark, df=silver_runways, target_table="dim_runways",
        strategy="upsert", pk_cols=["runway_sk"], cursor=master_cursor
    )
    insert_unknown_runway_sql = f"""
        INSERT INTO dim_runways (
            runway_sk, airport_sk, runway_name, true_heading, surface, has_lighting,
            is_closed, length_feet, width_feet, displaced_threshold_feet,
            latitude, longitude, effective_start_date, effective_end_date
        )
        VALUES (
            -1, -1, 'UNK', 0.0, 'Unknown', False,
            False, 0.0, 0.0, 0.0,
            0.0, 0.0, '1970-01-01', '9999-12-31'
        )
        ON CONFLICT (runway_sk) DO NOTHING;
    """
    master_cursor.execute(insert_unknown_runway_sql)

ERROR:root:Error in database: name 'silver_airports' is not defined
Traceback (most recent call last):
  File "/home/dottier/flights_etl/load/database.py", line 26, in get_db
    yield cur
  File "/home/dottier/flights_etl/load/database.py", line 49, in transaction_context
    yield new_cursor
  File "/tmp/ipykernel_1058/2310015354.py", line 6, in <module>
    spark=spark, df=silver_airports, target_table="dim_airports",
                    ^^^^^^^^^^^^^^^
NameError: name 'silver_airports' is not defined


In [3]:
from load.database import transaction_context

with transaction_context() as cur:
    cur.execute(
        "TRUNCATE TABLE gold.dim_airports, gold.dim_airlines, gold.dim_aircrafts, gold.dim_runways, gold.dim_flight_details, gold.dim_quality_combination, gold.fct_flights_intermediate;"
    )

In [35]:
silver_runways.filter("airport_sk = -1").show(50, truncate=False)

+-------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------------+------------------+
|runway_sk          |airport_sk|runway_name|true_heading|surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet|latitude|longitude|effective_start_date      |effective_end_date|
+-------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------------+------------------+
|8077286841863878235|-1        |UNKNOWN    |0.0         |Unknown|false       |false    |0.0        |0.0       |0.0                     |0.0     |0.0      |2025-10-13 16:49:22.618962|NULL              |
+-------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------------+----------

In [15]:
silver_flights.printSchema()

root
 |-- flight_composite_pk: string (nullable = true)
 |-- departure_airport_sk: long (nullable = true)
 |-- arrival_airport_sk: long (nullable = true)
 |-- airline_sk: long (nullable = true)
 |-- aircraft_sk: long (nullable = true)
 |-- departure_runway_sk: long (nullable = true)
 |-- arrival_runway_sk: long (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- flight_callsign: string (nullable = true)
 |-- flight_status: string (nullable = true)
 |-- codeshare_status: string (nullable = true)
 |-- is_cargo: boolean (nullable = true)
 |-- dep_scheduled_at_utc: timestamp (nullable = true)
 |-- dep_revised_at_utc: timestamp (nullable = true)
 |-- dep_runway_at_utc: timestamp (nullable = true)
 |-- dep_scheduled_at_local_ntz: timestamp (nullable = true)
 |-- dep_revised_at_local_ntz: timestamp (nullable = true)
 |-- dep_runway_at_local_ntz: timestamp (nullable = true)
 |-- dep_local_timezone: string (nullable = true)
 |-- arr_scheduled_at_utc: timestamp (nullable = true)


In [19]:
from pyspark.sql.functions import array_contains, concat, lit, col, array_join, format_string

#dim_flight_details_df = silver_flights.

# dim_flight_details_sql = f"""
#     CREATE TABLE dim_flight_details (
#         flight_details_sk SERIAL PRIMARY KEY,
#         flight_status TEXT NOT NULL,
#         codeshare_status TEXT NOT NULL,
#         is_cargo BOOLEAN NOT NULL,
        
#         -- This ensures we only store each unique combination once
#         UNIQUE (flight_status, codeshare_status, is_cargo)
#     );
# """

dim_flight_details_df = silver_flights.select(
    "flight_status", "codeshare_status", "is_cargo"
).distinct()

dim_quality_combination_df = silver_flights.select(
    array_contains("dep_quality", "Basic").alias("dep_has_basic"),
    array_contains("dep_quality", "Live").alias("dep_has_live"),
    array_contains("arr_quality", "Basic").alias("arr_has_basic"),
    array_contains("arr_quality", "Live").alias("arr_has_live"),
).distinct()

dim_quality_combination_df = dim_quality_combination_df.withColumn(
    "quality_desc",
    format_string(
        "Dep B:%s L:%s, Arr B:%s L:%s",
        col("dep_has_basic"), col("dep_has_live"),
        col("arr_has_basic"), col("arr_has_live")
    )
)

dim_quality_combination_df.show(10, truncate=False)


+-------------+------------+-------------+------------+---------------------------------------+
|dep_has_basic|dep_has_live|arr_has_basic|arr_has_live|quality_desc                           |
+-------------+------------+-------------+------------+---------------------------------------+
|true         |false       |true         |true        |Dep B:true L:false, Arr B:true L:true  |
|true         |true        |false        |false       |Dep B:true L:true, Arr B:false L:false |
|true         |true        |true         |true        |Dep B:true L:true, Arr B:true L:true   |
|true         |false       |false        |false       |Dep B:true L:false, Arr B:false L:false|
|false        |false       |true         |true        |Dep B:false L:false, Arr B:true L:true |
|false        |false       |true         |false       |Dep B:false L:false, Arr B:true L:false|
|true         |false       |true         |false       |Dep B:true L:false, Arr B:true L:false |
|true         |true        |true        

In [ ]:
fct_flights_df = (
    silver_flights
    .join(
        dim_flight_details_df, on=["flight_status", "codeshare_status", "is_cargo"], how="left"
    )
    .join(
        dim_quality_combination_df, on=["dep_quality", "arr_quality"], how="left"
    )
)

In [11]:
fct_flights_df.show(10, truncate=False)

+-------------+-------------+-------------+----------------+--------+---------------------------------------------------------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-----------------+-------------+---------------+--------------------+-------------------+-----------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+------------+--------+----------------+----------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+-------------+------------+-------------+------------+-----------------------------+
|dep_quality  |arr_quality  |flight_status|codeshare_status|is_cargo|flight_composite_pk                  

In [ ]:
with transaction_context() as cur:
    cur.execute("""
        ALTER TABLE dim_flight_details
        ADD 
    """)

ERROR:root:Error in database: syntax error at end of input
LINE 3:     
            ^
Traceback (most recent call last):
  File "/home/dottier/flights_etl/load/database.py", line 26, in get_db
    yield cur
  File "/home/dottier/flights_etl/load/database.py", line 49, in transaction_context
    yield new_cursor
  File "/tmp/ipykernel_1058/2660416548.py", line 2, in <module>
    cur.execute("""
psycopg2.errors.SyntaxError: syntax error at end of input
LINE 3:     
            ^



In [17]:
from load.utils import load_to_postgres, read_df_from_postgres
from load.database import transaction_context

with transaction_context() as master_cursor:
    load_to_postgres(
        spark=spark, df=dim_flight_details_df, target_table="dim_flight_details",
        strategy="upsert", pk_cols=["flight_status", "codeshare_status", "is_cargo"], cursor=master_cursor
    )

    load_to_postgres(
        spark=spark, df=dim_quality_combination_df, target_table="dim_quality_combination",
        strategy="upsert", pk_cols=["dep_has_basic", "dep_has_live", "arr_has_basic", "arr_has_live"], cursor=master_cursor
    )

In [ ]:
quality_lookup_df = read_df_from_postgres(spark, "dim_quality_combination")
flight_details_lookup_df = read_df_from_postgres(spark, "dim_flight_details")

quality_lookup_df.show(5)
flight_details_lookup_df.show(5)

+----------------+-------------+------------+-------------+------------+--------------------+
|quality_combo_sk|dep_has_basic|dep_has_live|arr_has_basic|arr_has_live|        quality_desc|
+----------------+-------------+------------+-------------+------------+--------------------+
|               1|         true|       false|         true|        true|Dep B:true L:fals...|
|               2|         true|        true|        false|       false|Dep B:true L:true...|
|               3|         true|        true|         true|        true|Dep B:true L:true...|
|               4|         true|       false|        false|       false|Dep B:true L:fals...|
|               5|        false|       false|         true|        true|Dep B:false L:fal...|
+----------------+-------------+------------+-------------+------------+--------------------+
only showing top 5 rows

+-----------------+-------------+----------------+--------+
|flight_details_sk|flight_status|codeshare_status|is_cargo|
+--------

In [22]:
fct_flights_df = silver_flights.withColumn(
    "dep_has_basic", array_contains(col("dep_quality"), "Basic")
).withColumn(
    "dep_has_live", array_contains(col("dep_quality"), "Live")
).withColumn(
    "arr_has_basic", array_contains(col("arr_quality"), "Basic")
).withColumn(
    "arr_has_live", array_contains(col("arr_quality"), "Live")
)

fct_flights_df = (
    fct_flights_df
    .join(
        flight_details_lookup_df,
        on=["flight_status", "codeshare_status", "is_cargo"],
        how="left"
    )
    .join(
        quality_lookup_df,
        on=["dep_has_basic", "dep_has_live", "arr_has_basic", "arr_has_live"],
        how="left"
    )
)

In [23]:
fct_flights_df.show(10)

+-------------+------------+-------------+------------+-------------+----------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-----------------+-------------+---------------+--------------------+-------------------+-----------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+-------------+------------+--------+----------------+----------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+-----------------+----------------+--------------------+
|dep_has_basic|dep_has_live|arr_has_basic|arr_has_live|flight_status|codeshare_status|is_cargo| flight_composite_pk|departure_air

In [ ]:
with transaction_context() as master_cursor:
    load_to_postgres(
        spark=spark, df=fct_flights_df, target_table="fct_flights",
        strategy="delete_insert", cursor=master_cursor
    )

In [14]:
import importlib
import load.utils  # make sure the module is imported
importlib.reload(load.utils)



ImportError: cannot import name 'transaction_context' from 'load.database' (/home/dottier/flights_etl/load/database.py)

In [3]:
silver_flights.printSchema()

root
 |-- flight_composite_pk: string (nullable = true)
 |-- departure_airport_sk: long (nullable = true)
 |-- arrival_airport_sk: long (nullable = true)
 |-- airline_sk: long (nullable = true)
 |-- aircraft_sk: long (nullable = true)
 |-- departure_runway_sk: long (nullable = true)
 |-- arrival_runway_sk: long (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- flight_callsign: string (nullable = true)
 |-- flight_status: string (nullable = true)
 |-- codeshare_status: string (nullable = true)
 |-- is_cargo: boolean (nullable = true)
 |-- dep_scheduled_at_utc: timestamp (nullable = true)
 |-- dep_revised_at_utc: timestamp (nullable = true)
 |-- dep_runway_at_utc: timestamp (nullable = true)
 |-- dep_scheduled_at_local_ntz: timestamp (nullable = true)
 |-- dep_revised_at_local_ntz: timestamp (nullable = true)
 |-- dep_runway_at_local_ntz: timestamp (nullable = true)
 |-- dep_local_timezone: string (nullable = true)
 |-- arr_scheduled_at_utc: timestamp (nullable = true)


In [6]:
import pyspark.sql.functions as sf
silver_flights.groupBy(sf.col("arr_quality")).agg(
    sf.count(sf.col("arr_quality")).alias("count")
).show()

+-------------+------+
|  arr_quality| count|
+-------------+------+
|           []|  7147|
|[Basic, Live]|133071|
|      [Basic]| 43538|
+-------------+------+



In [3]:
silver_flights.show(10, truncate=False)

+---------------------------------------------------------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-----------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+-----------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+-------------+------------+--------+----------------+----------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_composite_pk                                                  |departure_airport_sk|arrival_airport_sk  |airline_sk          |aircraft_sk         |departure_runway_sk|arrival_runway_

In [4]:
import pyspark.sql.functions as sf

silver_flights.groupBy(sf.col("flight_number")).agg(
    sf.count(sf.col("flight_number")).alias("count")
).orderBy(sf.col("count").desc()).show()

+-------------+-----+
|flight_number|count|
+-------------+-----+
|        HBJWC|   23|
|       UA 195|   18|
|       EK 206|   18|
|       ET 737|   18|
|       EK 255|   18|
|       EK 210|   18|
|       EK 209|   18|
|       UA 160|   18|
|        UA 45|   18|
|        UA 59|   18|
|        DL 21|   18|
|       AY 804|   18|
|       EK 256|   18|
|       ET 736|   18|
|        UA 41|   18|
|       AY 803|   18|
|        SQ 26|   18|
|        UA 31|   18|
|        SQ 25|   18|
|        AA 71|   17|
+-------------+-----+
only showing top 20 rows



In [14]:
silver_flights.filter("flight_number = 'EK 206'").show(10, truncate=False)

+-------------------------------------------------------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-----------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+-------------+------------+--------+----------------+----------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_composite_pk                                                |departure_airport_sk|arrival_airport_sk  |airline_sk          |aircraft_sk         |departure_runway_sk|arrival_runway_sk

In [12]:
silver_runways.filter("runway_sk = 8077286841863878235").show(20, truncate=False)

+-------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------------+------------------+
|runway_sk          |airport_sk|runway_name|true_heading|surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet|latitude|longitude|effective_start_date      |effective_end_date|
+-------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------------+------------------+
|8077286841863878235|-1        |UNKNOWN    |0.0         |Unknown|false       |false    |0.0        |0.0       |0.0                     |0.0     |0.0      |2025-10-13 16:49:22.618962|NULL              |
+-------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------------+----------

In [11]:
silver_airports.filter("airport_sk IN (-3869769487129258055, -7941055854170734276, 7883337853819725554 )").show(20, truncate=False)

+--------------------+------------+------------+-----------------+-----------------+------------+--------------+--------+---------+--------------+-----------------+
|airport_sk          |airport_iata|airport_icao|airport_name     |municipality_name|country_name|continent_name|latitude|longitude|elevation_feet|airport_time_zone|
+--------------------+------------+------------+-----------------+-----------------+------------+--------------+--------+---------+--------------+-----------------+
|-7941055854170734276|ZRH         |LSZH        |Zurich Kloten    |Zurich           |Switzerland |Europe        |47.4647 |8.549169 |1416.0        |Europe/Zurich    |
|-3869769487129258055|PMI         |LEPA        |Palma De Mallorca|Palma De Mallorca|Spain       |Europe        |39.5517 |2.73881  |27.0          |Europe/Madrid    |
|7883337853819725554 |BSL         |LFSB        |Bâle/Mulhouse    |NULL             |NULL        |NULL          |NULL    |NULL     |NULL          |Europe/Paris     |
+---------

In [8]:
silver_airlines.printSchema()
silver_aircrafts.printSchema()
silver_runways.printSchema()

root
 |-- airline_sk: long (nullable = true)
 |-- airline_iata: string (nullable = true)
 |-- airline_icao: string (nullable = true)
 |-- airline_name: string (nullable = true)

root
 |-- aircraft_sk: long (nullable = true)
 |-- aircraft_reg: string (nullable = true)
 |-- aircraft_mode_s: string (nullable = true)
 |-- aircraft_model: string (nullable = true)
 |-- aircraft_sk_type: string (nullable = true)

root
 |-- runway_sk: long (nullable = true)
 |-- airport_sk: long (nullable = true)
 |-- runway_name: string (nullable = true)
 |-- true_heading: double (nullable = true)
 |-- surface: string (nullable = true)
 |-- has_lighting: boolean (nullable = true)
 |-- is_closed: boolean (nullable = true)
 |-- length_feet: double (nullable = true)
 |-- width_feet: double (nullable = true)
 |-- displaced_threshold_feet: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- effective_start_date: timestamp (nullable = true)
 |-- effective_e

In [ ]:
with transaction_context() as cur:
    db.execute("""
        CREATE TABLE dim_runways (
            runway_sk BIGINT PRIMARY KEY,
            airport_sk BIGINT REFERENCES dim_airports (airport_sk),
            runway_name VARCHAR(100),
            true_heading DOUBLE PRECISION,
            surface VARCHAR(100),
            has_lighting BOOLEAN,
            is_closed BOOLEAN,
            length_feet DOUBLE PRECISION
            width_feet DOUBLE PRECISION,
            displaced_threshold_feet DOUBLE PRECISION,
            latitude DOUBLE PRECISION,
            longitude DOUBLE PRECISION,
            effective_start_Date TIMESTAMP,
            effective_end_date TIMESTAMP
        );
    """)

In [ ]:
with transaction_context() as db:
    # Print schema for a specific table, e.g., dim_airports
    table_name = 'dim_airports'
    db.execute("""
        SELECT column_name, data_type, is_nullable
        FROM information_schema.columns
        WHERE table_schema = 'public'
          AND table_name = %s;
    """, (table_name,))
    for col in db.fetchall():
        print(col)

In [3]:
silver_airports.filter("airport_iata = 'LHR'").show()

+--------------------+------------+------------+---------------+-----------------+--------------+--------------+--------+---------+--------------+-----------------+
|          airport_sk|airport_iata|airport_icao|   airport_name|municipality_name|  country_name|continent_name|latitude|longitude|elevation_feet|airport_time_zone|
+--------------------+------------+------------+---------------+-----------------+--------------+--------------+--------+---------+--------------+-----------------+
|-7580325845725858894|         LHR|        EGLL|London Heathrow|           London|United Kingdom|        Europe| 51.4706|-0.461941|          83.0|    Europe/London|
+--------------------+------------+------------+---------------+-----------------+--------------+--------------+--------+---------+--------------+-----------------+



In [4]:
from load.utils import load_to_postgres

load_to_postgres(spark, silver_airports, "dim_airports", "upsert", pk_cols=["airport_sk"])

In [ ]:
bronze_airports_df = read_df(spark, "bronze", "airports")
bronze_airports_df.show(40)

+----+----+---------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+------------+-----------------+--------------------+--------------------+--------------------+-------+
|icao|iata|localCode|           shortName|            fullName| municipalityName|            location|           elevation|             country|   continent|         timeZone|                urls|             runways|         currentTime|airport|
+----+----+---------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+------------+-----------------+--------------------+--------------------+--------------------+-------+
|LEMG| AGP|     NULL|              Málaga|              Málaga|           Málaga| {36.6749, -4.49911}|{16.15, 0.02, 0.0...|         {ES, Spain}|{EU, Europe}|    Europe/Madrid|{http://www.aena....|[{12, 117.7, {313...|{{2025-10-04 15:2...|    AGP|
|EHAM| AMS| 

: 

#### CSV testing

In [2]:
from pyspark.sql import DataFrame
from pathlib import Path

CSV_BASE_PATH = Path("/home/dottier/flights_etl/csv/")
def csv_to_df(spark: SparkSession, file_name: str) -> DataFrame:
    file_path = CSV_BASE_PATH / file_name
    return spark.read.csv(str(file_path), header=True, inferSchema=True)

In [3]:
dim_airports_csv = csv_to_df(spark, "airports.csv")
dim_runways_csv = csv_to_df(spark, "runways.csv")
dim_countries_csv = csv_to_df(spark, "countries.csv")
dim_regions_csv = csv_to_df(spark, "regions.csv")

In [ ]:


dim_airports_csv.filter(
    col("iata_code").isNotNull() | col("icao_code").isNotNull()
    # (col("type") == "large_airport") | (col("type") == "medium_airport")
).count()

10664

In [8]:
from pyspark.sql.functions import col

filtered_airports = dim_airports_csv.filter(
    col("iata_code").isNotNull() | col("icao_code").isNotNull()
)

In [ ]:
dim_runways_csv

In [25]:
dim_runways_csv.printSchema()

root
 |-- id: integer (nullable = true)
 |-- airport_ref: integer (nullable = true)
 |-- airport_ident: string (nullable = true)
 |-- length_ft: integer (nullable = true)
 |-- width_ft: integer (nullable = true)
 |-- surface: string (nullable = true)
 |-- lighted: integer (nullable = true)
 |-- closed: integer (nullable = true)
 |-- le_ident: string (nullable = true)
 |-- le_latitude_deg: double (nullable = true)
 |-- le_longitude_deg: double (nullable = true)
 |-- le_elevation_ft: integer (nullable = true)
 |-- le_heading_degT: double (nullable = true)
 |-- le_displaced_threshold_ft: integer (nullable = true)
 |-- he_ident: string (nullable = true)
 |-- he_latitude_deg: double (nullable = true)
 |-- he_longitude_deg: double (nullable = true)
 |-- he_elevation_ft: integer (nullable = true)
 |-- he_heading_degT: double (nullable = true)
 |-- he_displaced_threshold_ft: integer (nullable = true)



In [4]:
from pyspark.sql.functions import col, lit

# Common columns
common_cols = [
    col("airport_ident"),
    col("length_ft").alias("length_feet"),
    col("width_ft").alias("width_feet"),
    col("surface"),
    col("lighted"),
    col("closed"),
]

# Low-end columns
low_end_cols = common_cols + [
    col("le_ident").alias("runway_name"),
    col("le_latitude_deg").alias("latitude"),
    col("le_longitude_deg").alias("longitude"),
    col("le_elevation_ft").alias("elevation_feet"),
    col("le_heading_degT").alias("true_heading"),
    col("le_displaced_threshold_ft").alias("displaced_threshold_feet"),
    lit("LE").alias("runway_end_type")  # optional flag
]

# High-end columns
high_end_cols = common_cols + [
    col("he_ident").alias("runway_name"),
    col("he_latitude_deg").alias("latitude"),
    col("he_longitude_deg").alias("longitude"),
    col("he_elevation_ft").alias("elevation_feet"),
    col("he_heading_degT").alias("true_heading"),
    col("he_displaced_threshold_ft").alias("displaced_threshold_feet"),
    lit("HE").alias("runway_end_type")
]

# Build both DataFrames
low_end_runways = dim_runways_csv.select(*low_end_cols)
high_end_runways = dim_runways_csv.select(*high_end_cols)


In [8]:
low_end_runways.show(10, truncate=False)

25/11/05 15:02:26 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------+-----------+----------+-------+-------+------+-----------+------------------+-------------------+--------------+------------+------------------------+---------------+
|airport_ident|length_feet|width_feet|surface|lighted|closed|runway_name|latitude          |longitude          |elevation_feet|true_heading|displaced_threshold_feet|runway_end_type|
+-------------+-----------+----------+-------+-------+------+-----------+------------------+-------------------+--------------+------------+------------------------+---------------+
|00A          |80         |80        |ASPH-G |1      |0     |H1         |NULL              |NULL               |NULL          |NULL        |NULL                    |LE             |
|00AK         |2500       |70        |GRVL   |0      |0     |N          |NULL              |NULL               |NULL          |NULL        |NULL                    |LE             |
|00AL         |2300       |200       |TURF   |0      |0     |01         |NULL             

In [5]:
import pyspark.sql.functions as sf

union_runways = low_end_runways.unionByName(high_end_runways)
union_runways.groupBy("surface").agg(
    sf.count(sf.col("surface")).alias("count")
).orderBy(col("count").desc()).show()

+-------+-----+
|surface|count|
+-------+-----+
|    ASP|22576|
|   TURF|14972|
|    CON| 6982|
|   CONC| 6218|
|    GRS| 4394|
|   ASPH| 3276|
|    GRE| 3076|
|   Turf| 2366|
| TURF-G| 2028|
|    GVL| 2004|
|  WATER| 1316|
| ASPH-G| 1238|
|  Earth| 1126|
|    UNK| 1050|
|  Grass| 1034|
| TURF-F|  990|
|      G|  952|
|   DIRT|  918|
|      X|  842|
|      N|  702|
+-------+-----+
only showing top 20 rows



In [9]:
filtered_runways = (
    union_runways.join(
        filtered_airports.select(col("iata_code"), col("icao_code"), col("ident")),
        union_runways.airport_ident == filtered_airports.ident,
        how="inner"
    ).filter(
        # Business keys can't be null
        col("true_heading").isNotNull() &
        col("latitude").isNotNull() &
        col("longitude").isNotNull()
    )
)




In [ ]:
from pyspark.sql.functions import current_timestamp, lit, sha2, concat, concat_ws, lower, trim, round as spark_round, coalesce



# Standardize columns
dim_runways_df = filtered_runways.withColumn(
    "has_lighting", col("lighted") == 1
).withColumn(
    "is_closed", col("closed") == 1
).withColumn(
    "airport_bk",
    sha2(lower(trim(coalesce("iata_code", "icao_code"))), 256)
).withColumn(
    "runway_bk",
    sha2(
        concat_ws(
            "|",
            col("airport_bk"),
            spark_round(col("true_heading"), 1),
            spark_round(col("latitude"), 4),
            spark_round(col("longitude"), 4),
        ),
        256
    )
).withColumn(
    "_ingested_at", lit("batch_time")
).withColumn(
    "_inserted_at", current_timestamp()
)

dim_runways_df = dim_runways_df.drop(
    "lighted", "closed", "iata_code", "icao_code", "airport_ident", "ident"
)

from utils.expression_utils import get_select_expressions

# Apply columns order
select_exprs = get_select_expressions("silver", "dim_runways_ourairports")
dim_runways_df = dim_runways_df.select(*select_exprs)

In [15]:
from pyspark.sql.functions import col
dim_runways_df.filter((col("length_feet") == 4300) & (col("width_feet") == 150) & (col("runway_name") == "22") & (col("elevation_feet") == 692)).show(10)

+--------------------+--------------------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+------------------+------------------+--------------+---------------+------------+--------------------+
|           runway_bk|          airport_bk|runway_name|true_heading|surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet|          latitude|         longitude|elevation_feet|runway_end_type|_ingested_at|        _inserted_at|
+--------------------+--------------------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+------------------+------------------+--------------+---------------+------------+--------------------+
|fcd2a9f24818851b3...|f7f37497244f80513...|         22|       225.0|    CON|        true|     true|       4300|       150|                    NULL|35.666500091552734|-95.94650268554688|           692|             HE|  batch_time|2025-11-10 17:06:.

In [42]:
filtered_runways = (
    union_runways.join(
        filtered_airports.select(col("iata_code"), col("icao_code"), col("ident")),
        union_runways.airport_ident == filtered_airports.ident,
        how="inner"
    ).filter(
        col("true_heading").isNotNull() &
        col("latitude").isNotNull() &
        col("longitude").isNotNull()
    )
)

In [40]:
filtered_runways.printSchema()

root
 |-- airport_ident: string (nullable = true)
 |-- length_feet: integer (nullable = true)
 |-- width_feet: integer (nullable = true)
 |-- surface: string (nullable = true)
 |-- lighted: integer (nullable = true)
 |-- closed: integer (nullable = true)
 |-- runway_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elevation_feet: integer (nullable = true)
 |-- true_heading: double (nullable = true)
 |-- displaced_threshold_feet: integer (nullable = true)
 |-- runway_end_type: string (nullable = false)
 |-- id: integer (nullable = true)
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- latitude_deg: double (nullable = true)
 |-- longitude_deg: double (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: st

In [ ]:
from pyspark.sql.functions import (
    sha2, lower, trim, coalesce, current_timestamp,
    concat_ws, col, lit
) 
from datetime import datetime, timezone

batch_time = datetime.now(timezone.utc)

dim_airports = filtered_airports.withColumn(
    "airport_sk",
    sha2(lower(trim(coalesce("iata_code", "icao_code"))), 256)
).withColumn(
    "scheduled_service", col("scheduled_service") == "yes"
)

data_cols = [c for c in dim_airports.columns if c != "airport_sk"]

dim_airports = dim_airports.withColumn(
    "_data_hash_ourairports",
    sha2(
        concat_ws(
            "|",
            *[coalesce(lower(trim(col(c))), lit("NULL")) for c in data_cols]
        ),
    256)
).withColumn(
    "_ingested_at", lit(batch_time)
).withColumn(
    "_inserted_at", current_timestamp()
)

dim_airports = dim_airports.drop(
    "id", "ident", "continent", "iso_region",
    "home_link", "wikipedia_link", "keywords"
)

In [24]:
filtered_airports.count()

10664

In [39]:
filtered_runways.count()

17328

In [23]:
dim_runways.show(50)

+-------------+-----------+----------+-----------+-------+------+-----------+-------------------+-------------------+--------------+------------+------------------------+---------------+------------+---------+
|airport_ident|length_feet|width_feet|    surface|lighted|closed|runway_name|           latitude|          longitude|elevation_feet|true_heading|displaced_threshold_feet|runway_end_type|has_lighting|is_closed|
+-------------+-----------+----------+-----------+-------+------+-----------+-------------------+-------------------+--------------+------------+------------------------+---------------+------------+---------+
|         07FA|       4500|        70|       ASPH|      1|     0|         04|  25.32069969177246|  -80.2791976928711|          NULL|        40.0|                    NULL|             LE|        true|    false|
|         0CO2|       4000|        40|       ASPH|      1|     0|         11| 38.854400634765625|-106.93800354003906|          8980|       121.1|               

In [36]:
dim_airports.show(10, truncate=False)

+-------------+-----------------------------+---------------+----------------+------------+-----------+---------------+-----------------+---------+---------+--------+----------+----------------------------------------------------------------+----------------------------------------------------------------+-------------------------+--------------------------+
|type         |name                         |latitude_deg   |longitude_deg   |elevation_ft|iso_country|municipality   |scheduled_service|icao_code|iata_code|gps_code|local_code|airport_sk                                                      |_data_hash_openflights                                          |_ingested_at             |_inserted_at              |
+-------------+-----------------------------+---------------+----------------+------------+-----------+---------------+-----------------+---------+---------+--------+----------+----------------------------------------------------------------+------------------------------------

In [31]:
dim_airports.filter(col("iso_country").isNull()).count()

0

In [53]:
from pyspark.sql.functions import round as spark_round

dim_runways = filtered_runways.withColumn(
    "has_lighting", col("lighted") == 1
).withColumn(
    "is_closed", col("closed") == 1
).withColumn(
    "airport_sk",
    sha2(lower(trim(coalesce("iata_code", "icao_code"))), 256)
).withColumn(
    "runway_sk",
    sha2(
        concat_ws(
            "|",
            col("airport_sk"),
            spark_round(col("true_heading"), 1),
            spark_round(col("latitude"), 4),
            spark_round(col("longitude"), 4),
        ),
        256
    )
)

dim_runways = dim_runways.drop(
    "lighted", "closed", "iata_code", "icao_code", "airport_ident", "ident"
)

In [ ]:
runways_data_cols = [c for c in dim_runways.columns if c != "runway_sk"]

dim_runways = dim_runways.withColumn(
    "_data_hash_ourairports",
    sha2(
        concat_ws(
            "|",
            *[coalesce(lower(trim(col(c))), lit("NULL")) for c in runways_data_cols]
        ),
    256)
).withColumn(
    "_ingested_at", lit(batch_time)
).withColumn(
    "_inserted_at", current_timestamp()
)

In [55]:
dim_runways.show(50)

+-----------+----------+-----------+-----------+-------------------+-------------------+--------------+------------+------------------------+---------------+------------+---------+--------------------+--------------------+----------------------+--------------------+--------------------+
|length_feet|width_feet|    surface|runway_name|           latitude|          longitude|elevation_feet|true_heading|displaced_threshold_feet|runway_end_type|has_lighting|is_closed|          airport_sk|           runway_sk|_data_hash_openflights|        _ingested_at|        _inserted_at|
+-----------+----------+-----------+-----------+-------------------+-------------------+--------------+------------+------------------------+---------------+------------+---------+--------------------+--------------------+----------------------+--------------------+--------------------+
|       4500|        70|       ASPH|         04|  25.32069969177246|  -80.2791976928711|          NULL|        40.0|                    

In [52]:
dim_runways.printSchema()

root
 |-- length_feet: integer (nullable = true)
 |-- width_feet: integer (nullable = true)
 |-- surface: string (nullable = true)
 |-- runway_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elevation_feet: integer (nullable = true)
 |-- true_heading: double (nullable = true)
 |-- displaced_threshold_feet: integer (nullable = true)
 |-- runway_end_type: string (nullable = false)
 |-- has_lighting: boolean (nullable = true)
 |-- is_closed: boolean (nullable = true)
 |-- airport_sk: string (nullable = true)
 |-- runway_sk: string (nullable = true)
 |-- _data_hash_openflights: string (nullable = true)
 |-- _ingested_at: timestamp (nullable = false)
 |-- _inserted_at: timestamp (nullable = false)



In [56]:
dim_countries_csv.show(10, truncate=False)

+------+----+--------------------+---------+--------------------------------------------------+--------------------------------------+
|id    |code|name                |continent|wikipedia_link                                    |keywords                              |
+------+----+--------------------+---------+--------------------------------------------------+--------------------------------------+
|302672|AD  |Andorra             |EU       |https://en.wikipedia.org/wiki/Andorra             |Andorran airports                     |
|302618|AE  |United Arab Emirates|AS       |https://en.wikipedia.org/wiki/United_Arab_Emirates|UAE,مطارات في الإمارات العربية المتحدة|
|302619|AF  |Afghanistan         |AS       |https://en.wikipedia.org/wiki/Afghanistan         |NULL                                  |
|302722|AG  |Antigua and Barbuda |NA       |https://en.wikipedia.org/wiki/Antigua_and_Barbuda |Antiguan airports                     |
|302723|AI  |Anguilla            |NA       |https://en.

In [ ]:
dim_regions = (
    dim_regions_csv.alias("regions")
    .join(
        dim_countries_csv.alias("countries")
        .select(
            col("code").alias("country_code"),
            col("name").alias("country_name")
        ),
        col("regions.iso_country") == col("country_code"),
        how="inner"
    )
)

dim_regions = dim_regions.drop("id", "wikipedia_link", col("country_code"), "keywords")

regions_data_cols = [c for c in dim_regions.columns if c != "regions.code"]

dim_regions = dim_regions.withColumn(
    "_data_hash_ourairports",
    sha2(
        concat_ws(
            "|",
            *[coalesce(lower(trim(col(c))), lit("NULL")) for c in regions_data_cols]
        ),
    256)
).withColumn(
    "_ingested_at", lit(batch_time)
).withColumn(
    "_inserted_at", current_timestamp()
)

root
 |-- code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- name: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- country_name: string (nullable = true)



In [80]:
dim_regions.show(10, truncate=False)
dim_regions.filter(col("regions.code").isNull()).count()

+------+----------+--------------------------+---------+-----------+--------------------+----------------------------------------------------------------+-------------------------+--------------------------+
|code  |local_code|name                      |continent|iso_country|country_name        |_data_hash_openflights                                          |_ingested_at             |_inserted_at              |
+------+----------+--------------------------+---------+-----------+--------------------+----------------------------------------------------------------+-------------------------+--------------------------+
|AD-02 |02        |Canillo Parish            |EU       |AD         |Andorra             |4c8db59073b3973257eeb768c14b9fdad5a7316922e0f3e43b070072e8ac6bae|2025-11-05 17:13:02.70852|2025-11-05 18:25:22.233611|
|AD-03 |03        |Encamp Parish             |EU       |AD         |Andorra             |b8ac1f6c5d89aa3e9f6ab55d5175cc377a35bc475ac18fb32975c493ae68b077|2025-11-05 17:

0

In [57]:
print(dim_countries_csv.count())
print(dim_regions_csv.count())

249
3941


In [8]:
from pyspark.sql.functions import col

dim_countries_csv.join(
    dim_regions_csv,
    on=dim_countries_csv.code == dim_regions_csv.iso_country,
    how="anti"
).show()

+---+----+----+---------+--------------+--------+
| id|code|name|continent|wikipedia_link|keywords|
+---+----+----+---------+--------------+--------+
+---+----+----+---------+--------------+--------+

